# Summary

Make sure updated BackendSelector class is working properly. DO NOT PUSH THIS TO GITHUB (added to gitignore so api keys are not exposed).

UPDATE: renamed to GPTBackend.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import openai
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    GPTBackend, PromptManager, ConversationManager, load_api_key, HF_API_KEY
from htools import *

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.


In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [4]:
gpt = GPTBackend()
gpt

GPTBackend <current_name: openai>

In [5]:
gpt.ls()


Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>


In [6]:
for name in gpt.name2key:
    print(name)
    gpt.ls()
    with gpt(name):
        gpt.ls()
        assert gpt.key() == gpt.name2key[name]
    gpt.ls()
    print(spacer())

openai

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>
Switching openai backend to "openai".

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>
Switching  backend back to "openai".

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>

-------------------------------------------------------------------------------

gooseai

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>
Switching openai backend to "gooseai".

Base: https://api.goose.ai/v1
Query func: <function query_gpt3 at 0x1202be158>
Switching  backend back to "openai".

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>

-------------------------------------------------------------------------------

huggingface

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>
Switching openai backend to "huggingface".

Base: https://api.openai.com
Query func: <function query_gpt

In [7]:
for name in gpt.name2key:
    gpt.switch(name)
    gpt.ls()
    assert gpt.key() == gpt.name2key[name]
    print(spacer())

Switching openai backend to "openai".

Base: https://api.openai.com
Query func: <function query_gpt3 at 0x1202be158>

-------------------------------------------------------------------------------

Switching openai backend to "gooseai".

Base: https://api.goose.ai/v1
Query func: <function query_gpt3 at 0x1202be158>

-------------------------------------------------------------------------------

Switching openai backend to "huggingface".

Base: https://api.goose.ai/v1
Query func: <function query_gpt_huggingface at 0x1202be048>

-------------------------------------------------------------------------------

Switching openai backend to "hobby".

Base: https://api.goose.ai/v1
Query func: <function query_gpt_j at 0x12023fea0>

-------------------------------------------------------------------------------

Switching openai backend to "repeat".

Base: https://api.goose.ai/v1
Query func: <function query_gpt_repeat at 0x1202be0d0>

-----------------------------------------------------------

In [8]:
gpt.ls()


Base: https://api.goose.ai/v1
Query func: <function query_gpt_banana at 0x1202be2f0>


In [10]:
gpt.query(prompt='How historically accurate is')

('How historically accurate is', 'How historically accurate is')

In [11]:
gpt.query(prompt='How historically accurate is', engine_i=3)

('How historically accurate is', 'How historically accurate is')

In [15]:
gpt.query(prompt='How historically accurate is',
          log_path='data/logs/test.json', engine_i=3)

('How historically accurate is', 'How historically accurate is')

In [16]:
gpt.switch('hobby')
gpt.query('How accurate is')

Switching openai backend to "hobby".


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:69: UserWarning: GPT-J api does not support other kwargs: {'engine_i': 0, 'frequency_penalty': 0.0}
  warnings.warn(f'GPT-J api does not support other kwargs: {kwargs}')


MockFunctionException: HTTPConnectionPool(host='api.vicgalle.net', port=5000): Max retries exceeded with url: /generate?context=How+accurate+is&token_max_length=50&temperature=0.7&top_p=1.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12a0b95c0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [9]:
gpt.switch('gooseai')
gpt.ls()
gpt.query('test', max_tokens=4)

In [18]:
gpt.query('test', max_tokens=4, log_path='data/logs/test.json')

('test', '_set_max')

## Test PromptManager and ConversationManager

In [16]:
conv = ConversationManager(verbose=False)

In [17]:
prompter = PromptManager()

simplify_ml: This uses the expensive davinci model and doesn't work so well without it. Temperature is set to 0.3 but this hasn't been extensively tuned.
-------------------------------------------------------------------------------

shortest: This prompt takes no input.
-------------------------------------------------------------------------------

extract_backend_slot: WARNING: This seemed to work okay when testing in the gooseai console but further testing suggests it needs more work.
-------------------------------------------------------------------------------

ml_abstract: I haven't tuned hyperparameters much - even engine_i=1 sometimes provides good resuls but it seems a bit inconsistent.
-------------------------------------------------------------------------------

translate: User input should consist of 1 line like 'Translate to German:' followed by a second line with a sentence in English. You may want to use a variable number of tokens defined as a function of the input

In [21]:
for name in gpt.name2key:
    gpt.switch(name)
    kwargs_ = prompter.kwargs('translate')
    print(kwargs_)
    assert kwargs_['mock_func'] == gpt.name2mock[name]
    print(spacer())

Switching openai backend to "openai".
{'engine_i': 3, 'temperature': 0.3, 'frequency_penalty': 0.3, 'max_tokens': 200, 'logprobs': None, 'stream': False, 'mock': False, 'return_full': False, 'strip_output': True, 'mock_func': None, 'mock_mode': 'raise', 'stop': ['Translate to ', 'Translation:']}

-------------------------------------------------------------------------------

Switching openai backend to "gooseai".
{'engine_i': 3, 'temperature': 0.3, 'frequency_penalty': 0.3, 'max_tokens': 200, 'logprobs': None, 'stream': False, 'mock': False, 'return_full': False, 'strip_output': True, 'mock_func': None, 'mock_mode': 'raise', 'stop': ['Translate to ', 'Translation:']}

-------------------------------------------------------------------------------

Switching openai backend to "huggingface".
{'engine_i': 3, 'temperature': 0.3, 'frequency_penalty': 0.3, 'max_tokens': 200, 'logprobs': None, 'stream': False, 'mock': False, 'return_full': False, 'strip_output': True, 'mock_func': <function 

In [22]:
for name in gpt.name2key:
    gpt.switch(name)
    kwargs_ = conv.kwargs('Albert Einstein')
    print(kwargs_)
    assert kwargs_['mock_func'] == gpt.name2mock[name]
    print(spacer())

Switching openai backend to "openai".
{'engine_i': 3, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': False, 'mock': False, 'return_full': False, 'strip_output': True, 'mock_func': None, 'mock_mode': 'raise', 'stop': ['\n\nMe:', 'This is a conversation with']}

-------------------------------------------------------------------------------

Switching openai backend to "gooseai".
{'engine_i': 3, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': False, 'mock': False, 'return_full': False, 'strip_output': True, 'mock_func': None, 'mock_mode': 'raise', 'stop': ['\n\nMe:', 'This is a conversation with']}

-------------------------------------------------------------------------------

Switching openai backend to "huggingface".
{'engine_i': 3, 'temperature': 0.5, 'frequency_penalty': 0.1, 'max_tokens': 250, 'logprobs': None, 'stream': False, 'mock': False, 'return_full': False, 'strip_output': True, 'mock

In [25]:
for name in gpt.name2key:
    # Broken at the moment.
    if name == 'hobby': continue
    gpt.switch(name)
    with conv.converse('Albert Einstein'):
        res = conv.query(
            'Hi Albert. What is a book you\'d recommend?',
            max_tokens=4,
            engine_i=0
        )
    print(res[1])
    print(spacer())

Switching openai backend to "openai".
A book, a

-------------------------------------------------------------------------------

Switching openai backend to "gooseai".
I think the best

-------------------------------------------------------------------------------

Switching openai backend to "huggingface".
I would recommend a

-------------------------------------------------------------------------------

Switching openai backend to "repeat".
This is a conversation with Albert Einstein. Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely acknowledged to be one of the greatest physicists of all time. Einstein is known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics.

Me: Hi Albert. What is a book you'd recommend?

Albert Einstein:

-------------------------------------------------------------------------------



In [24]:
gpt.switch('repeat')
with conv.converse('Albert Einstein'):
    res = conv.query(
        'Hi Albert. What is a book you\'d recommend?',
        max_tokens=4,
        engine_i=0
    )
    print(res[1])

Switching openai backend to "repeat".
This is a conversation with Albert Einstein. Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely acknowledged to be one of the greatest physicists of all time. Einstein is known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics.

Me: Hi Albert. What is a book you'd recommend?

Albert Einstein:


## Truncation

In [436]:
def truncate_at_first_stop(text, stop_phrases, finish_reason='',
                           trunc_full=True, trunc_partial=True):
    if trunc_full:
        idx = [idx for idx in map(text.find, stop_phrases) if idx >= 0]
        stop_idx = min(idx or [None])
        text = text[:stop_idx]

    # If the completion was cut short due to length AND the completion ends
    # with the majority of a stop phrase, we infer that this should be stripped
    # from the end. This rule won't be perfect but it seems like a decent bet.
    if trunc_partial and finish_reason == 'length':
        for phrase in stop_phrases:
            chunk = phrase[:max(int(round(.8 * len(phrase))), 4)]
            if text.endswith(chunk):
                warnings.warn(
                    'Guessing that truncation is reasonable because '
                    'finish_reason="length" and completion ends with a '
                    'partial stop phrase.'
                )
                return text.rpartition(chunk)[0]
    return text

In [437]:
mock = load(C.mock_stream_paths[False])
mock.keys()

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.


dict_keys(['id', 'object', 'created', 'model', 'choices'])

In [438]:
mock.choices[0].finish_reason

'length'

In [459]:
no_stop = """
Einstein: How are you?"""

full_stop = """
Einstein: How are you?

Me:"""

extra_stop = """
Einstein: How are you?

Me: I am good."""

partial_stop = """
Einstein: How are you?

Me"""

tricky_stop = """
Einstein: How are you?

Me, I'm fine.
"""

correct = no_stop

In [461]:
assert truncate_at_first_stop(no_stop, conv._kwargs['stop']) == correct

In [462]:
assert truncate_at_first_stop(full_stop, conv._kwargs['stop']) == correct

In [463]:
assert truncate_at_first_stop(extra_stop, conv._kwargs['stop']) == correct
assert truncate_at_first_stop(extra_stop, conv._kwargs['stop'], 
                              finish_reason='length') == correct

In [467]:
assert truncate_at_first_stop(partial_stop, 
                              conv._kwargs['stop']) == partial_stop
assert truncate_at_first_stop(partial_stop, conv._kwargs['stop'],
                              finish_reason='length',
                              trunc_partial=False) == partial_stop
assert truncate_at_first_stop(partial_stop, conv._kwargs['stop'],
                              finish_reason='length') == correct

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Guessing that truncation is reasonable because finish_reason="length" and completion ends with a partial stop phrase.
  app.launch_new_instance()


In [466]:
assert truncate_at_first_stop(tricky_stop, 
                              conv._kwargs['stop']) == tricky_stop
assert truncate_at_first_stop(tricky_stop, conv._kwargs['stop'],
                              finish_reason='length',
                              trunc_partial=False) == tricky_stop
assert truncate_at_first_stop(tricky_stop, conv._kwargs['stop'],
                              finish_reason='length') == tricky_stop

In [39]:
gpt.switch('gooseai')
conv.start_conversation('Albert Einstein')
res = conv.query(
    'Hi Albert.',
    max_tokens=50,
    engine_i=0,
    return_full=True
)
print(conv.full_conversation())

Switching openai backend to "gooseai".
This is a conversation with Albert Einstein. Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely acknowledged to be one of the greatest physicists of all time. Einstein is known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics.

Me: Hi Albert.

Albert Einstein: Hi.

Me:


In [40]:
res_strict_limit = conv.query(
    'What open question in physics do you most hope to learn an answer to?',
    max_tokens=1,
    engine_i=0,
    return_full=True
)
print(conv.full_conversation())

This is a conversation with Albert Einstein. Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely acknowledged to be one of the greatest physicists of all time. Einstein is known for developing the theory of relativity, but he also made important contributions to the development of the theory of quantum mechanics.

Me: Hi Albert.

Albert Einstein: Hi.

Me:

Me: What open question in physics do you most hope to learn an answer to?

Albert Einstein: Well


In [41]:
res[1]

'Hi.\n\nMe:'

In [43]:
res_strict_limit[1]

'Well'

In [44]:
res[2]['choices'][0]['finish_reason']

'stop'

In [45]:
res_strict_limit[2]['choices'][0]['finish_reason']

'length'

In [86]:
truncate_at_first_stop(res[1], conv.kwargs()['stop'], res[2])

'Hi.'

In [87]:
truncate_at_first_stop(res_strict_limit[1], conv.kwargs()['stop'],
                       res_strict_limit[2])

'Well'

In [90]:
text = 'Well\n\nMe'
truncate_at_first_stop(text, conv.kwargs()['stop'],
                       res_strict_limit[2])

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Guessing that truncation is reasonable because finish_reason="length" and completion ends with a partial stop phrase.
  app.launch_new_instance()


'Well'

In [72]:
stops = conv.kwargs()['stop'] + ['\n\nAlbert Einstein:', 
                                 '\n\nJeremy Howard:',
                                 '\n\nI:']
stops

['\n\nMe:',
 'This is a conversation with',
 '\n\nAlbert Einstein:',
 '\n\nJeremy Howard:',
 '\n\nI:']

In [73]:
{stop: stop[:max(int(round(.8 * len(stop))), 4)] for stop in stops}

{'\n\nMe:': '\n\nMe',
 'This is a conversation with': 'This is a conversation',
 '\n\nAlbert Einstein:': '\n\nAlbert Einst',
 '\n\nJeremy Howard:': '\n\nJeremy Howa',
 '\n\nI:': '\n\nI:'}

In [91]:
with gpt('huggingface'):
    res_hf = gpt.query('Who are you?', engine_i=0, return_full=True)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Switching openai backend to "huggingface".


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:153: UserWarning: query_gpt_huggingface received unused kwargs {'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'logprobs': None, 'n': 1, 'stream': False, 'logit_bias': None}.
  warnings.warn('query_gpt_huggingface received unused kwargs '


Switching  backend back to "gooseai".


In [94]:
res_hf[2]

<OpenAIObject text_completion id=cmpl-2onIQZc241dqfnum3Uducepxxa39u at 0x12862f200> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": {
        "text_offset": [
          147,
          149,
          150,
          152,
          153,
          155,
          156,
          157,
          162,
          163,
          166,
          167,
          169,
          170,
          174,
          175,
          181,
          182,
          190,
          193,
          194,
          199,
          200,
          201,
          206,
          207,
          212,
          213,
          219,
          220,
          225,
          226,
          227,
          232,
          233,
          236,
          237,
          239,
          240,
          244,
          245,
          251,
          252,
          255,
          256,
          258,
          259,
          263,
          264,
          265
        ],
        "token_logprobs": [
   

## Check for "finish_reason"

Do HF/vic galle backends provide finish_reason in their response?

(These are very messy versions - the cleaner final versions are in jabberwocky.opena_utils.)

Thoughts on desired interface. np refers to number of prompts, nc refers to number of completions.

```
# np>1, nc=1
prompt_, response_ = qpt.query(['a', 'b', 'c'])
prompt_ # ['a', 'b', 'c']
response_ # ['after a', 'after b', 'after c']

# np=1, nc=1
prompt_, response_ = gpt.query('a')
prompt_ # 'a'                 ['a']
response_ # 'after a'         [['a']]

# np=1, nc>1
prompt_, response_ = gpt.query('a', n=2)
prompt_ # 'a'                                      ['a']
response_ # ['after a 1', 'after a 2']             [['after a 1', 'after a 2']]

# np>1, nc>1
prompt_, response_ = gpt.query(['a', 'b'], n=2)
prompt_ # ['a', 'b']
response_ # [['after a 1', 'after a 2'], ['after b 1', 'after b2']]
```

In [380]:
import requests

from jabberwocky.openai_utils import MockFunctionException, query_gpt_banana
from jabberwocky.utils import with_signature

In [469]:
def squeeze(*args, n=1):
    """Return either the input `args` or the first item of each arg, depending
    on our choice of n. We effectively treat n as a boolean. This is used by
    the query_gpt_{} functions to return either a (str, dict) tuple or a 
    (list[str], list[dict]) tuple, depending on the number of completions
    we ask for.
    """
    return tuple(arg[0] for arg in args) if n == 1 else args

In [393]:
@valuecheck
def wip_query_gpt_huggingface(
        prompt, engine_i=0, temperature=1.0, repetition_penalty=None,
        max_tokens=50, top_k=None, top_p=None, n=1, **kwargs
):
    """Query EleuetherAI gpt models using the Huggingface API. This was called
    query_gpt_neo in a former version of the library (which is used by the
    GUI) but the API now hosts a GPT-J model as well so I renamed it.

    Parameters
    ----------
    prompt: str
    engine_i: int
        Determines which Huggingface model API to query. See
        config.C.backend_engines['huggingface'].
        Those names refer to the number of
        parameters in the model, where bigger models generally produce higher
        quality results but may be slower (in addition to the actual inference
        being slower to produce, the better models are also more popular so the
        API is hit with more requests).
    temperature: float
        Between 0 and 1. 0-0.4 is good for straightforward informational
        queries (e.g. reformatting, writing business emails) while 0.7-1 is
        good for more creative works. Warning: huggingface docs say this
        actually goes from 0-100 - should check if they're using this value
        differently than the openai API.
    top_k: None or int
        Kind of like top_p in that smaller values may produce more
        sensible but less creative responses. While top_p limits options to
        a cumulative percentage, top_k limits it to a discrete number of
        top choices.
    top_p: None or float
        Value in [0.0, 1.0] if provided. Kind of like temperature in that
        smaller values may produce more sensible but less creative responses.
    repetition_penalty
    max_tokens: int
        Sets max response length. One token is ~.75 words.
    kwargs: any
        Just lets us absorb extra kwargs when used in place of query_gpt3().

    Returns
    -------
    # TODO update
    # Currently returns List[str], List[dict] where index i in each list gives
    # us the ith completion. We always pass in a single prompt.
    
    tuple or iterator: When stream=False, we return a tuple where the first
    item is the prompt (str) and the second is the response text(str). If
    return_full is True, a third item consisting of the whole response object
    is returned as well. When stream=True, we return an iterator where each
    step contains a single token. This will either be the text response alone
    (str) or a tuple of (text, response) if return_full is True. Unlike in
    non-streaming mode, we don't return the prompt - that seems less
    appropriate for many time steps.
    
    Returns
    -------
    tuple[str]: Prompt, response tuple, just like query_gpt_3().
    """
    if not isinstance(prompt, str):
        raise TypeError(f'Prompt must be str, not {type(prompt)}.')
    
    # Hardcode backend in case we use this function outside of the 
    # GPTBackend.query wrapper.
    engine = GPTBackend.engine(engine_i, backend='huggingface')

    # Docs say we can return up to 256 tokens but API sometimes throws errors
    # if we go above 250.
    headers = {'Authorization':
               f'Bearer api_{HF_API_KEY}'}
    # Notice the names don't always align with parameter names - I wanted
    # those to be more consistent with query_gpt3() function. Also notice
    # that types matter: if Huggingface expects a float but gets an int, we'll
    # get an error.
    if repetition_penalty is not None:
        repetition_penalty = float(repetition_penalty)
    stop = tolist(kwargs.pop('stop', []))
    if kwargs:
        warnings.warn('query_gpt_huggingface received unused kwargs '
                      f'{kwargs}.')

    data = {'inputs': prompt,
            'parameters': {'top_k': top_k, 'top_p': top_p,
                           'temperature': float(temperature),
                           'max_new_tokens': min(max_tokens, 250),
                           'repetition_penalty': repetition_penalty,
                           'return_full_text': False,
                           'num_return_sequences': n}}
    url = f'https://api-inference.huggingface.co/models/EleutherAI/{engine}'
    try:
        # Put the request itself inside try too in case of timeout.
        r = requests.post(url, headers=headers, data=json.dumps(data))
        r.raise_for_status()
    except requests.HTTPError as e:
        raise MockFunctionException(str(e)) from None

    res = r.json()
    # Structure: text, full response
    # List[str], List[dict]
    return [row['generated_text'] for row in res], res
    


#     # TODO: r is a list of dicts with only 1 key ('generated_text'). Can't
#     # get logprobs. Experimenting with making second item in returned val be
#     # a list of completions in case n > 1.

#     # Huggingface doesn't natively provide the `stop` parameter that OpenAI
#     # does so we have to do this manually.
#     completions = [row['generated_text'] for row in r.json()]
#     if stop:
#         truncated = []
#         for completion in completions:
#             idx = [idx for idx in map(completion.find, stop) if idx >= 0]
#             stop_idx = min(idx or [None])
#             truncated.append(completion[:stop_idx])
#         completions = truncated
#     return prompt, completions

In [398]:
# API still down, can't test this atm.
def wip_query_gpt_j(prompt, temperature=0.7, max_tokens=50, **kwargs):
    """Queries free GPT-J API. GPT-J has 6 billion parameters and is, roughly
    speaking, the open-source equivalent of Curie (3/19/22 update: size sounds
    more like Babbage actually). It was trained on more
    code than GPT3 though so it may do surprisingly well at those kinds of
    tasks. This function should be usable as a mock_func argument in
    query_gpt_3.

    API uptime may be questionable though. There's an accompanying front end
    here:
    http://api.vicgalle.net:8000/

    Parameters
    ----------
    prompt: str
    temperature: float
    max_tokens: int
    kwargs: any
        Only supported options are top_p (float) and stop (Iterable[str]).
        Notice that stream mode is not supported.

    Returns
    -------
    tuple[str, dict]: Response text, full response dict.
    """
    params = {'context': prompt,
              'token_max_length': max_tokens,
              'temperature': temperature,
              'top_p': kwargs.pop('top_p', 1.0)}

    # Ensure that we end up with a list AND that stop is still Falsy if user
    # explicitly passes in stop=None.
    stop = tolist(kwargs.pop('stop', None) or [])
    if stop: params['stop_sequence'] = stop[0]

    # Must keep this after the block of stop-related logic above.
    if kwargs:
        warnings.warn(f'GPT-J api does not support other kwargs: {kwargs}')

    try:
        res = requests.post('http://api.vicgalle.net:5000/generate',
                            params=params)
        res.raise_for_status()
    except Exception as e:
        raise MockFunctionException(str(e)) from None
    res = res.json()
    return res['text'], res

#     # Endpoint doesn't support multiple stop sequences so we have to
#     # postprocess. Even with a single stop sequence, it includes it while gpt3
#     # and my gpt-neo function exclude it, so we need to handle that here.
#     idx = min([i for i in map(res['text'].find, stop) if i >= 0] or [None])
#     completion = res['text'][:idx]
#     return res['prompt'], completion

In [434]:
def wip_query_gpt3(prompt, engine_i=0, temperature=0.7, top_p=1.0,
               frequency_penalty=0.0, presence_penalty=0.0,
               max_tokens=50, logprobs=None, n=1, stream=False,
               logit_bias=None, **kwargs):
    """Convenience function to query gpt3. Mostly serves 2 purposes:
    1. Build in some mocking functionality for cheaper/free testing.
    2. Explicitly add some parameters and descriptions to the function
    docstring, since openai.Completion.create does not include most kwargs.

    Parameters
    ----------
    prompt: str
    engine_i: int
        Corresponds to engines defined in config, where 0 is the cheapest, 3
        is the most expensive, etc.
    temperature: float
        Between 0 and 1. 0-0.4 is good for straightforward informational
        queries (e.g. reformatting, writing business emails) while 0.7-1 is
        good for more creative works.
    top_p: float
        Value in (0.0, 1.0] that limits the model to sample from tokens making
        up the top_p percent combined. I.e. higher values allow for more
        creativity (like high temperature) and low values are closer to argmax
        sampling (like low temperature). API recommends setting a sub-maximal
        value for at most one of this and temperature, not both.
    frequency_penalty: float
        Value in [-2.0, 2.0] where larger (more positive) values more heavily
        penalize words that have already occurred frequently in the text.
        Usually reasonable to keep this in [0, 1].
    presence_penalty: float
        Value in [-2.0, 2.0] where larger (more positive) values more heavily
        penalize words that have already occurred in the text. Usually
        reasonable to keep this in [0, 1].
    max_tokens: int
        Sets max response length. One token is ~.75 words.
    logprobs: int or None
        Get log probabilities for top n candidates at each time step. This
        will only be useful if you set return_full=True.
    n: int
        Number of possible completions to return. Careful: values > 1 can add
        up quickly w.r.t. cost.
    stream: bool
        If True, return an iterator instead of a str/tuple. See the returns
        section as the output is slightly different. I believe each chunk
        returns one token when stream is True.
    logit_bias: dict or None
        If provided, should map string(s) (NUMERIC INDEX of word tokens,
        not the tokens themselves) to ints between -100 and 100 (inclusive?).
        Values in (-1, 1) should be used to nudge the model, while larger
        values can effectively ban/compel the model to use certain words.
    kwargs: any
        Additional kwargs to pass to gpt3. Should rarely be necessary.

    Returns
    -------
    tuple or iterator: When stream=False, we return a tuple where the first
    item is the prompt (str) and the second is the response text(str). If
    return_full is True, a third item consisting of the whole response object
    is returned as well. When stream=True, we return an iterator where each
    step contains a single token. This will either be the text response alone
    (str) or a tuple of (text, response) if return_full is True. Unlike in
    non-streaming mode, we don't return the prompt - that seems less
    appropriate for many time steps.
    """
    if not isinstance(prompt, str):
        raise TypeError(f'Prompt must be str, not {type(prompt)}.')
    if stream and n > 1:
        raise RuntimeError('Stream=True and n>1 not supported.')
        
#     if stream and strip_output:
#         warnings.warn('strip_output is automatically set to False when stream '
#                       'is True. It would be impossible to correctly '
#                       'reconstruct outputs otherwise.')
    if temperature < 1 and top_p < 1:
        warnings.warn('You set both temperature and top_p to values < 1. '
                      'API recommends setting only one of these to '
                      'sub-maximal value.')
#     if logprobs and not return_full:
#         warnings.warn('You set logprobs to a nonzero value but '
#                       'return_full=False. If you want to access the logprobs, '
#                       'you should set return_full=True.')

    res = openai.Completion.create(
        engine=GPTBackend.engine(engine_i, 'openai'),
        prompt=prompt,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        max_tokens=max_tokens,
        logprobs=logprobs,
        n=n,
        stream=stream,
        logit_bias=logit_bias or {},
        **kwargs
    )
        
    # Extract text and return. Zip maintains lazy evaluation.
    if stream:
        # Each item in zipped object is (str, dict-like).
        texts = (chunk.choices[0].text for chunk in res)
        chunks = (dict(chunk.choices[0]) for chunk in res)
        return zip(texts, chunks)
    
    # Structure: (List[str], List[dict-like])
    return [row.text for row in res.choices], \
        [dict(choice) for choice in res.choices]

In [407]:
@with_signature(wip_query_gpt3, keep=True)
@mark(requires_stopword_truncation=True)
def wip_query_gpt_goose(prompt, **kwargs):
    return wip_query_gpt3(prompt, **kwargs)

In [394]:
prompts = [
    'Who is the', 
    ('I am a', 'You are a'),
]
ns = [1, 3]
kwargs = DotDict(
    zip(
        ['single_both', 'multi_out', 'multi_in', 'multi_both'],
        [{'prompt': p, 'n': n, 'max_tokens': 5} 
         for p in prompts for n in ns]
    )
)

In [395]:
hf = DotDict()
for k, v in tqdm(kwargs.items()):
    try:
        hf[k] = wip_query_gpt_huggingface(**v, engine_i=1) 
    except Exception as e:
        print(k, e)

  0%|          | 0/4 [00:00<?, ?it/s]

multi_in Prompt must be str, not <class 'tuple'>.
multi_both Prompt must be str, not <class 'tuple'>.


In [396]:
hf.single_both

([' President? When is the'], [{'generated_text': ' President? When is the'}])

In [397]:
hf.multi_out

([' real “Man of', ' most memorable character from the', ' real Trump? (and'],
 [{'generated_text': ' real “Man of'},
  {'generated_text': ' most memorable character from the'},
  {'generated_text': ' real Trump? (and'}])

In [433]:
with gpt('openai'):
    for tok, data in wip_query_gpt3(**kwargs.single_both, logprobs=3, 
                                    engine_i=0, stream=True):
        print(tok)
        print(data)
        print(spacer())

Switching openai backend to "openai".
 most
{'text': ' important', 'index': 0, 'logprobs': <OpenAIObject at 0x1706c01a8> JSON: {
  "text_offset": [
    15
  ],
  "token_logprobs": [
    -2.2002852
  ],
  "tokens": [
    " important"
  ],
  "top_logprobs": [
    {
      " important": -2.2002852,
      " likely": -2.7987826,
      " popular": -3.2215893
    }
  ]
}, 'finish_reason': None}

-------------------------------------------------------------------------------

 player
{'text': ' for', 'index': 0, 'logprobs': <OpenAIObject at 0x16fe24728> JSON: {
  "text_offset": [
    32
  ],
  "token_logprobs": [
    -2.2986562
  ],
  "tokens": [
    " for"
  ],
  "top_logprobs": [
    {
      " for": -2.2986562,
      " in": -0.9703081,
      " on": -1.8214434
    }
  ]
}, 'finish_reason': None}

-------------------------------------------------------------------------------

Switching  backend back to "gooseai".


In [427]:
g = DotDict()
with gpt('openai'):
    g['single_both'] = wip_query_gpt3(**kwargs.single_both, logprobs=3, engine_i=0)
    g['multi_out'] = wip_query_gpt3(**kwargs.multi_out, logprobs=3, engine_i=0)

Switching openai backend to "openai".
Switching  backend back to "gooseai".


In [428]:
g.single_both

([' President of the United States'],
 [{'text': ' President of the United States',
   'index': 0,
   'logprobs': <OpenAIObject at 0x1706c0308> JSON: {
     "text_offset": [
       10,
       20,
       23,
       27,
       34
     ],
     "token_logprobs": [
       -4.364347,
       -0.7310643,
       -0.6073798,
       -1.8252875,
       -0.011561717
     ],
     "tokens": [
       " President",
       " of",
       " the",
       " United",
       " States"
     ],
     "top_logprobs": [
       {
         " author": -3.6692274,
         " best": -3.04953,
         " most": -3.2980042
       },
       {
         " of": -0.7310643,
         "?": -1.7396235,
         "?\"": -2.611957
       },
       {
         " The": -4.221831,
         " the": -0.6073798,
         " this": -4.6789083
       },
       {
         " Board": -3.5698235,
         " United": -1.8252875,
         " University": -3.6078427
       },
       {
         " Kingdom": -5.8680987,
         " Nations": -6.1417484,

In [432]:
[row['logprobs']['top_logprobs'] for row in g.multi_out[1]]

[[<OpenAIObject at 0x1706c0468> JSON: {
    " author": -3.6692274,
    " best": -3.04953,
    " most": -3.2980042
  },
  <OpenAIObject at 0x1706c0570> JSON: {
    " of": -0.39385706,
    "?": -1.6094236,
    "?\"": -3.9906347
  },
  <OpenAIObject at 0x1706c05c8> JSON: {
    " \"": -3.2908247,
    " the": -1.431546,
    " this": -0.88533527
  },
  <OpenAIObject at 0x1706c06d0> JSON: {
    " article": -1.0468491,
    " book": -2.1007652,
    " post": -3.169608
  },
  <OpenAIObject at 0x1706c0728> JSON: {
    " and": -3.2372837,
    ",": -4.04183,
    "?": -0.15100385
  }],
 [<OpenAIObject at 0x1706c0830> JSON: {
    " author": -3.6692274,
    " best": -3.04953,
    " most": -3.2980042
  },
  <OpenAIObject at 0x1706c0888> JSON: {
    " character": -0.8551719,
    " protagonist": -3.3901486,
    " villain": -3.3227801
  },
  <OpenAIObject at 0x1706c08e0> JSON: {
    " in": -1.4130918,
    " of": -1.8321267,
    "?": -1.2511454
  },
  <OpenAIObject at 0x1706c0938> JSON: {
    " the": -1.151

In [224]:
query_gpt_banana('Horologist')

('Horologist',
 "\n\nA horologist is a watchmaker or watchmaker's apprentice who is trained in horology, the study of watches.  The horologist is responsible for repairing watches, making and maintaining a watchmaker's tools, and for maintaining the quality")

In [205]:
tmp_ = load(C.mock_stream_paths[True])
len(tmp_)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.


5

In [209]:
tmp_[0].choices[0].text

'<PROMPT> make'

In [210]:
tmp_[1].choices[0].text

' a'

In [154]:
res30 = wip_query_gpt_huggingface('Who is the',
                                  n=2,
                                  engine_i=1,
                                  max_tokens=10)

In [155]:
res30

('Who is the',
 [' most dangerous cat to dogcat people?\n\n',
  ' true heir of David, the Goliath of G'])

In [196]:
res30_arr = wip_query_gpt_huggingface(
    ['Who is the', 'I am the'],
    n=1,
    engine_i=1,
    max_tokens=5
)

In [198]:
res30_arr.json()

[[{'generated_text': ' Father?\n \n'}],
 [{'generated_text': ' head of a local chapter'}]]

In [229]:
res30_arr.json()

[[{'generated_text': ' Father?\n \n'}],
 [{'generated_text': ' head of a local chapter'}]]

In [199]:
res30_multi = wip_query_gpt_huggingface(
    ['Who is the', 'I am the'],
    n=2,
    engine_i=1,
    max_tokens=5
)

In [200]:
res30_multi.json()

[[{'generated_text': ' most evil person of the'},
  {'generated_text': ' most qualified on these forums'}],
 [{'generated_text': ' owner, or an authorized'},
  {'generated_text': ' author of the books "'}]]

**gptj (broken)**

In [230]:
res30_j = wip_query_gpt_j(
    'Who is the', max_tokens=10
)

MockFunctionException: HTTPConnectionPool(host='api.vicgalle.net', port=5000): Max retries exceeded with url: /generate?context=Who+is+the&token_max_length=10&temperature=0.7&top_p=1.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x128705e48>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [175]:
# res30_o for date, openai. 
# Seeing what response obj looks like w/ >1 completions.
with gpt('gooseai'):
    res30_o = wip_query_gpt3(
        'Who is the', max_tokens=10, engine_i=0, n=2, logprobs=5
    )

Switching openai backend to "gooseai".


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: UserWarning: You set logprobs to a nonzero value but return_full=False. If you want to access the logprobs, you should set return_full=True.


Switching  backend back to "gooseai".


In [176]:
len(res30_o['choices'])

2

In [178]:
for row in res30_o['choices']:
    print(row.finish_reason)
    print(row.text)
    print(row.logprobs.top_logprobs)
    print(spacer())

length
 real winner in the new law?

The
[<OpenAIObject at 0x12891be60> JSON: {
  " best": -2.83984375,
  " first": -3.68359375,
  " man": -4.12890625,
  " most": -3.193359375,
  " real": -3.591796875
}, <OpenAIObject at 0x12891b048> JSON: {
  " culprit": -4.57421875,
  " enemy": -4.1796875,
  " victim": -4.39453125,
  " winner": -3.578125,
  " \ufffd": -4.46875
}, <OpenAIObject at 0x12891beb8> JSON: {
  " and": -3.1484375,
  " here": -1.97265625,
  " in": -1.162109375,
  " of": -1.4462890625,
  "?": -2.232421875
}, <OpenAIObject at 0x12891be08> JSON: {
  " a": -4.12890625,
  " all": -2.935546875,
  " the": -0.90625,
  " these": -4.71484375,
  " this": -1.40234375
}, <OpenAIObject at 0x12891bf10> JSON: {
  " Brexit": -3.87109375,
  " US": -4.32421875,
  " current": -4.07421875,
  " new": -4.39453125,
  " war": -4.07421875
}, <OpenAIObject at 0x12891bf68> JSON: {
  " U": -4.31640625,
  " US": -4.26953125,
  " tax": -3.32421875,
  " world": -3.41015625,
  " \ufffd": -4.1328125
}, <OpenAI

In [179]:
# arr refers to passing in array of prompts.
# Seeing what response obj looks like w/ >1 completions.
with gpt('gooseai'):
    res30_arr = wip_query_gpt3(
        ['The', 'The quick', 'The quick brown', 'The quick brown fox'],
        max_tokens=1, engine_i=0, n=1, logprobs=5
    )

Switching openai backend to "gooseai".


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: UserWarning: You set logprobs to a nonzero value but return_full=False. If you want to access the logprobs, you should set return_full=True.


Switching  backend back to "gooseai".


In [182]:
len(res30_arr.choices)

4

In [184]:
for row in res30_arr.choices:
    print(row.finish_reason)
    print(row.text)
    print(row.logprobs.top_logprobs)
    print(spacer())

length
 invention
[<OpenAIObject at 0x12892f728> JSON: {
  " authors": -3.5859375,
  " content": -4.5625,
  " following": -4.3984375,
  " invention": -3.349609375,
  " present": -2.2734375
}]

-------------------------------------------------------------------------------

length
 brown
[<OpenAIObject at 0x12892f938> JSON: {
  " and": -2.09375,
  " answer": -2.45703125,
  " brown": -3.111328125,
  ",": -3.083984375,
  "-": -2.646484375
}]

-------------------------------------------------------------------------------

length
 fox
[<OpenAIObject at 0x12892fb48> JSON: {
  " (": -5.3203125,
  " Fox": -4.2734375,
  " and": -5.39453125,
  " f": -5.171875,
  " fox": -0.1324462890625
}]

-------------------------------------------------------------------------------

length
 jumped
[<OpenAIObject at 0x12892fc50> JSON: {
  " is": -4.96484375,
  " jumped": -1.75,
  " jumps": -0.259033203125,
  ",": -5.37890625,
  "bytes:'\\n'": -5.57421875
}]

--------------------------------------------------

**Mutiple prompts AND multiple completions.**

In [185]:
# arr refers to passing in array of prompts.
# Seeing what response obj looks like w/ >1 completions.
with gpt('gooseai'):
    res30_multi = wip_query_gpt3(
        ['The', 'The quick', 'The quick brown'],
        max_tokens=1, engine_i=0, n=2, logprobs=5
    )

Switching openai backend to "gooseai".


/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: UserWarning: You set logprobs to a nonzero value but return_full=False. If you want to access the logprobs, you should set return_full=True.


Switching  backend back to "gooseai".


In [187]:
len(res30_multi.choices)

6

In [188]:
# Looks like outer loop is determined by n prompts and inner loops is
# determined by n completions.
for row in res30_multi.choices:
    print(row.finish_reason)
    print(row.text)
    print(row.logprobs.top_logprobs)
    print(spacer())

length
 present
[<OpenAIObject at 0x128566410> JSON: {
  " authors": -3.5859375,
  " content": -4.5625,
  " following": -4.3984375,
  " invention": -3.349609375,
  " present": -2.2734375
}]

-------------------------------------------------------------------------------

length
 present
[<OpenAIObject at 0x1285664c0> JSON: {
  " authors": -3.5859375,
  " content": -4.5625,
  " following": -4.3984375,
  " invention": -3.349609375,
  " present": -2.2734375
}]

-------------------------------------------------------------------------------

length
 and
[<OpenAIObject at 0x128566150> JSON: {
  " and": -2.09375,
  " answer": -2.45703125,
  " brown": -3.111328125,
  ",": -3.083984375,
  "-": -2.646484375
}]

-------------------------------------------------------------------------------

length
,
[<OpenAIObject at 0x128566570> JSON: {
  " and": -2.09375,
  " answer": -2.45703125,
  " brown": -3.111328125,
  ",": -3.083984375,
  "-": -2.646484375
}]

------------------------------------------

In [192]:
n_prompts = 3
n_completions = 2
[[row.text for row in res30_multi.choices[i:i + n_completions]]
 for i in range(0, n_prompts * n_completions, n_completions)]

[[' present', ' present'], [' and', ','], [' fox', ' fox']]